# Decorator 디자인 패턴

In [1]:
def hello():
    print('--- hello 시작 ---')  # 로그 기능
    print('안녕...')  # hello 함수의 주된 기능
    print('--- hello 종료 ---')  # 로그 기능

In [2]:
hello()

--- hello 시작 ---
안녕...
--- hello 종료 ---


In [3]:
def world():
    print('--- world 시작 ---')
    print('World...')
    print('--- world 종료 ---')

In [4]:
world()

--- world 시작 ---
World...
--- world 종료 ---


hello() 함수와 world() 함수는 주된 기능만을 작성, 부가적인 기능(로깅, 아규먼트 검사, ...)들은 공통으로 작성하는 기법.

In [5]:
def trace(func):
    # 지역 함수(local function) 선언
    def wrapper():
        print(f'===== {func.__name__} 시작 =====')
        func()  # trace 함수의 아규먼트인 함수 func를 실행.
        print(f'===== {func.__name__} 종료 =====')

    return wrapper  # 지역 함수 "객체"를 리턴.

In [6]:
def hello():
    print('안녕...')

In [7]:
def world():
    print('World...')

In [8]:
hello_trace = trace(hello)

In [9]:
type(hello_trace)

function

In [10]:
hello_trace()

===== hello 시작 =====
안녕...
===== hello 종료 =====


In [11]:
world_trace = trace(world)
world_trace()

===== world 시작 =====
World...
===== world 종료 =====


trace() 함수를 데코레이터(decorator) 함수라고 함.

`@decorator` 애너테이션을 호출하고자 하는 함수 선언 부분에 사용하면 더 편리하게 이용할 수 있음.

In [16]:
@trace
def hello2():
    print('hello2...')

In [17]:
hello2()

===== hello2 시작 =====
hello2...
===== hello2 종료 =====


In [18]:
@trace
def world2():
    print('world2...')

In [19]:
world2()

===== world2 시작 =====
world2...
===== world2 종료 =====


함수에 데코레이터를 여러 개 지정할 수도 있음.

In [20]:
def deco1(func):
    def wrapper():
        print('--- deco1 시작 ---')
        func()
        print('--- deco1 종료 ---')

    return wrapper

In [21]:
def deco2(func):
    def wrapper():
        print('--- deco2 시작 ---')
        func()
        print('--- deco2 종료 ---')

    return wrapper

In [22]:
@deco1
@deco2
def hello_world():
    print('Hello, world!')

In [23]:
hello_world()

--- deco1 시작 ---
--- deco2 시작 ---
Hello, world!
--- deco2 종료 ---
--- deco1 종료 ---


# 파라미터와 반환값을 처리하는 데코레이터

In [24]:
def add(x, y):
    return x + y

In [25]:
def trace(func):  # 데코레이터에서 호출할 함수를 아규먼트로 전달받음
    def wrapper(x, y):  # 호출할 함수 func()과 동일하게 파라미터들을 선언.
        r = func(x, y)  # 함수 func에게 파라미터 x와 y를 아규먼트로 전달하면서 호출.
        print(f'{func.__name__}(x={x}, y={y}) -> {r}')
        return r  # func의 리턴 값을 반환.

    return wrapper

In [26]:
@trace
def add(x, y):
    return x + y

In [27]:
result = add(1, 2)

add(x=1, y=2) -> 3


In [28]:
print(result)

3


In [29]:
@trace
def subtract(x, y):
    return x - y

In [30]:
result = subtract(1, 2)

subtract(x=1, y=2) -> -1


In [31]:
print(result)

-1


## 가변길이 인수 함수 데코레이터

In [32]:
def trace(func):  # 실제로 호출할 함수를 아규먼트로 전달받음
    def wrapper(*args, **kwargs):  # 가변길이 인수로 파라미터를 선언.
        r = func(*args, **kwargs)  # 함수 func에게 args와 kwargs를 "unpacking"해서 아규먼트로 전달.
        print(f'{func.__name__}(args={args}, kwargs={kwargs}) -> {r}')
        return r  # func의 리턴 값을 반환.

    return wrapper

In [33]:
@trace
def get_max(*args):
    return max(args)

In [34]:
result = get_max(10, 20, 0, 1, 100, 55)

get_max(args=(10, 20, 0, 1, 100, 55), kwargs={}) -> 100


In [35]:
print(result)

100


In [36]:
result = get_max(1, 10, -1)

get_max(args=(1, 10, -1), kwargs={}) -> 10


In [37]:
print(result)

10


## unpacking 연산자

In [41]:
def add_three_number(x, y, z):
    return x + y + z

In [42]:
add_three_number(1, 2, 3)

6

In [44]:
try:
    r = add_three_number((1, 2, 3))
    print(r)
except Exception as e:
    print(e)

add_three_number() missing 2 required positional arguments: 'y' and 'z'


In [45]:
try:
    r = add_three_number(*(1, 2, 3))  # *(1, 2, 3) -> 1, 2, 3
    print(r)
except Exception as e:
    print(e)

6


In [47]:
def print_info(name, age):
    print(f'이름: {name}, 나이: {age}')

In [49]:
print_info(name='오쌤', age=16)

이름: 오쌤, 나이: 16


In [50]:
try:
    print_info({'name': '홍길동', 'age': 20})
except Exception as e:
    print(e)

print_info() missing 1 required positional argument: 'age'


In [51]:
try:
    print_info(**{'name': '홍길동', 'age': 20})
    # unpacking 연산자: **{key1: value1, key2: value2} -> key1=value1, key2=value2
except Exception as e:
    print(e)

이름: 홍길동, 나이: 20


# 파라미터를 갖는 데코레이터 작성

In [52]:
def is_multiple(n):  # 파라미터를 갖는 데코레이터
    def decorator(func):  # 호출할 함수를 아규먼트로 전달받음.
        def wrapper(x, y):  # 파라미터 x, y - 호출할 함수 func의 아규먼트로 사용.
            r = func(x, y)
            print(f'{func.__name__}(x={x}, y={y}) -> {r}')
            if r % n == 0:
                print(f'{func.__name__}의 리턴 값은 {n}의 배수입니다.')
            else:
                print(f'{func.__name__}의 리턴 값은 {n}의 배수가 아닙니다.')
            return r

        return wrapper

    return decorator

In [53]:
@is_multiple(2)
def add(x, y):
    return x + y

In [54]:
result = add(1, 2)

add(x=1, y=2) -> 3
add의 리턴 값은 2의 배수가 아닙니다.


In [55]:
print(result)

3


In [56]:
result = add(10, 20)

add(x=10, y=20) -> 30
add의 리턴 값은 2의 배수입니다.


In [57]:
print(result)

30


In [58]:
@is_multiple(3)
def subtract(x, y):
    return x - y

In [59]:
result = subtract(10, 1)

subtract(x=10, y=1) -> 9
subtract의 리턴 값은 3의 배수입니다.


In [60]:
print(result)

9


In [61]:
@is_multiple(2)
@is_multiple(3)
def add(x, y):
    return x + y

In [62]:
result = add(2, 4)

add(x=2, y=4) -> 6
add의 리턴 값은 3의 배수입니다.
wrapper(x=2, y=4) -> 6
wrapper의 리턴 값은 2의 배수입니다.


In [63]:
print(result)

6


데코레이터(`@decorator`)를 여러 개 사용하면 데코레이터에서 반환된 wrapper 함수가 다른 데코레이터의 아규먼트로 전달됨. 그래서 두번째 데코레이터에서 `__name__` 속성의 값은 `wrapper`가 됨.

처음에 전달된 함수 이름을 그대로 출력하고 싶을 때 `functools` 모듈의 `wraps` 데코레이터를 사용하면 됨.

In [64]:
import functools

In [65]:
def is_multiple(n):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(x, y):
            r = func(x, y)
            print(f'{func.__name__}(x={x}, y={y}) -> {r}')
            if r % n == 0:
                print(f'{func.__name__}의 리턴 값은 {n}의 배수입니다.')
            else:
                print(f'{func.__name__}의 리턴 값은 {n}의 배수가 아닙니다.')

            return r

        return wrapper

    return decorator

In [66]:
@is_multiple(2)
@is_multiple(3)
def subtract(x, y):
    return x - y

In [67]:
result = subtract(10, 1)

subtract(x=10, y=1) -> 9
subtract의 리턴 값은 3의 배수입니다.
subtract(x=10, y=1) -> 9
subtract의 리턴 값은 2의 배수가 아닙니다.


# 클래스로 데코레이터 작성

In [70]:
class Square:
    def __init__(self):
        print('Square 객체 생성됨.')

    def __call__(self, n):
        return n ** 2

In [71]:
square = Square()  # 생성자 호출 -> __init__ 메서드 호출

Square 객체 생성됨.


In [72]:
square(3)  # 인스턴스를 함수처럼 호출 -> __call__ 메서드 호출

9